In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages t|o load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
# import xgboost as xgb
from xgboost import XGBClassifier 
import warnings 
import optuna
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
sample_submission=pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
train=pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')

In [4]:
train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [5]:
train.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [6]:
train.shape

(20758, 18)

In [7]:
test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [8]:
test.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS'],
      dtype='object')

In [9]:
test.shape

(13840, 17)

In [10]:
cat_cols = train.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('NObeyesdad')
cat_cols

['Gender',
 'family_history_with_overweight',
 'FAVC',
 'CAEC',
 'SMOKE',
 'SCC',
 'CALC',
 'MTRANS']

In [11]:
num_cols = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_cols.remove('id') 
num_cols

['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [12]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

column_trans = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    remainder= 'passthrough',
    verbose_feature_names_out=False,
)

In [13]:
scaler = StandardScaler()

In [14]:
X = train[num_cols + cat_cols]
y = train['NObeyesdad']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20,stratify=y, random_state=42)

In [15]:
X.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS
0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,Male,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation
1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,Female,yes,yes,Frequently,no,no,no,Automobile
2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,Female,yes,yes,Sometimes,no,no,no,Public_Transportation
3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,Female,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation
4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,Male,yes,yes,Sometimes,no,no,Sometimes,Public_Transportation


In [16]:
operations = [
    ("OneHotEncoder", column_trans),
    ('Scaler', scaler),
    ("RF_model", RandomForestClassifier(n_estimators=1000,
        criterion='log_loss',
        min_samples_leaf=2,
        max_depth=20,
        min_samples_split=6,
        class_weight="balanced", random_state=42))
#         ('rf', RandomForestClassifier(n_estimators=344,criterion='log_loss',min_samples_leaf=2,max_depth=95, min_samples_split=6, random_state=42)),
]

pipe_model_rf = Pipeline(steps=operations)

pipe_model_rf.fit(X_train, y_train)

#     ("RF_model", RandomForestClassifier(max_depth=20, criterion='entropy',max_features=25,min_samples_split=20,n_estimators=1000,class_weight="balanced", random_state=42)),


Pipeline(steps=[('OneHotEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'CAEC', 'SMOKE',
                                                   'SCC', 'CALC', 'MTRANS'])],
                                   verbose_feature_names_out=False)),
                ('Scaler', StandardScaler()),
                ('RF_model',
                 RandomForestClassifier(class_weight='balanced',
                                        criterion='log_loss', max_depth=20,
                                        min_samples_leaf=2, min_samples_split=6,
                                        n_estimators=1000, random_state=42))])

In [17]:
y_pred = pipe_model_rf.predict(X_val)

In [18]:
# Print the performance metrics
print(classification_report(y_val, y_pred))

accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

                     precision    recall  f1-score   support

Insufficient_Weight       0.92      0.94      0.93       505
      Normal_Weight       0.86      0.86      0.86       617
     Obesity_Type_I       0.89      0.89      0.89       582
    Obesity_Type_II       0.96      0.97      0.97       650
   Obesity_Type_III       1.00      1.00      1.00       809
 Overweight_Level_I       0.81      0.74      0.77       485
Overweight_Level_II       0.78      0.83      0.80       504

           accuracy                           0.90      4152
          macro avg       0.89      0.89      0.89      4152
       weighted avg       0.90      0.90      0.90      4152

Validation Accuracy: 0.8990847784200385


# Here

In [19]:
# Predict on the test set
# test_predictions = pipe_model_rf.predict(test[num_cols + cat_cols])

In [20]:
# # Prepare a submission file
# submission = pd.DataFrame({'id': test['id'], 'NObeyesdad': test_predictions})
# submission.to_csv('submission.csv', index=False)

In [21]:
# xg_operations = [
#     ("OneHotEncoder", column_trans),
#     ('Scaler', scaler),
#     ("XG_model", XGBClassifier(max_depth=5, n_estimators=140, learning_rate=0.25, class_weight="balanced", random_state=42)),
# ]

# pipe_model_xg = Pipeline(steps=xg_operations)


# pipe_model_xg.fit(X_train, y_train)

In [22]:
# y_pred_xg = pipe_model_xg.predict(X_val)


In [23]:
# # Print the performance metrics
# print(classification_report(y_val, y_pred_xg))

# accuracy = accuracy_score(y_val, y_pred_xg)
# print(f'Validation Accuracy: {accuracy}')

In [24]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, num_cols),
        ('cat', column_trans, cat_cols)
    ])

In [25]:
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# from sklearn.ensemble import VotingClassifier
# # Define the base models
# model1 = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', XGBClassifier(max_depth=5,n_estimators=140, random_state=42))
# ])

# model2 = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', LGBMClassifier(max_depth=5,random_state=42))
# ])

# model3 = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', CatBoostClassifier(max_depth=5, random_state=42,verbose=False))  
# ])

# model4 = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('rf', RandomForestClassifier(n_estimators=1000,
#         criterion='log_loss',
#         min_samples_leaf=2,
#         max_depth=5,
#         min_samples_split=6,
#         class_weight="balanced", random_state=42))
# ])
# # Combine all models using soft voting
# combined_model = VotingClassifier(
#     estimators=[('xgb', model1), ('lgbm', model2), ('cat', model3), ('rf', model4)], 
#     voting='soft'
# )

# # Train the combined model
# combined_model.fit(X_train, y_train)

# # Validate the model
# y_pred = combined_model.predict(X_val)

# # Print the performance metrics
# print(classification_report(y_val, y_pred))

# accuracy = accuracy_score(y_val, y_pred)
# print(f'Validation Accuracy: {accuracy}')



In [26]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
# Define the base models
model1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(max_depth=5, random_state=42))
])

model2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(random_state=42))
])

model3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(random_state=42,verbose=False))  
])

model4 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(max_depth=5, random_state=42))
])
# Combine all models using soft voting
combined_model = VotingClassifier(
    estimators=[('xgb', model1), ('lgbm', model2), ('cat', model3), ('rf', model4)], 
    voting='soft'
)

# Train the combined model
combined_model.fit(X_train, y_train)

# Validate the model
y_pred = combined_model.predict(X_val)

# Print the performance metrics
print(classification_report(y_val, y_pred))

accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2042
[LightGBM] [Info] Number of data points in the train set: 16606, number of used features: 30
[LightGBM] [Info] Start training from score -2.107657
[LightGBM] [Info] Start training from score -1.907572
[LightGBM] [Info] Start training from score -1.964755
[LightGBM] [Info] Start training from score -1.855022
[LightGBM] [Info] Start training from score -1.635117
[LightGBM] [Info] Start training from score -2.146046
[LightGBM] [Info] Start training from score -2.107657
                     precision    recall  f1-score   support

Insufficient_Weight       0.94      0.95      0.95       505
      Normal_Weight       0.88      0.90      0.89       617
     Obesity_Type_I       0.88      0.89      0.89       582
    Obesity_Type_II     

In [27]:
# Predict on the test set
test_predictions = combined_model.predict(test[num_cols + cat_cols])



# Prepare a submission file
submission = pd.DataFrame({'id': test['id'], 'NObeyesdad': test_predictions})
submission.to_csv('submission.csv', index=False)